In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import datetime
import arrow
import os
import pandas as pd
import warnings
import time

from util.file_util import StockTwitsFileReader
from util.stocktwits_helper import (
    get_est_market_datetime, get_daily_sentiment_count_df,
    get_all_stocktwits_tickers)

In [4]:
from arrow.factory import ArrowParseWarning

In [5]:
# Disable Arrow warnings per https://github.com/crsmithdev/arrow/issues/612
warnings.simplefilter("ignore", ArrowParseWarning)

In [6]:
stock_twits_reader = StockTwitsFileReader()

In [7]:
ticker = 'SHAK'

In [8]:
start_date = '2018-01-01'
end_date = '2018-03-30'

In [9]:
df = stock_twits_reader.read_twit_file_in_range(ticker, start_date, end_date, cols='default')

In [11]:
daily_sentiment_df = get_daily_sentiment_count_df(df)

In [12]:
df[['date_est', 'created_at_est', 'mkt_date']].head(30)

,date_est,created_at_est,mkt_date
0,2018-01-01,2018-01-01 21:14:58,2018-01-02
1,2018-01-01,2018-01-01 09:10:05,2018-01-01
2,2018-01-01,2018-01-01 06:48:49,2018-01-01
3,2018-01-02,2018-01-02 23:43:30,2018-01-03
4,2018-01-02,2018-01-02 21:44:31,2018-01-03
5,2018-01-02,2018-01-02 21:43:55,2018-01-03
6,2018-01-02,2018-01-02 21:43:48,2018-01-03
7,2018-01-02,2018-01-02 18:42:51,2018-01-03
8,2018-01-02,2018-01-02 15:51:06,2018-01-02
9,2018-01-02,2018-01-02 11:42:02,2018-01-02


In [13]:
daily_sentiment_df.head(10)

,Bearish,Bullish
mkt_date,,
2018-01-03,4.0,1.0
2018-01-04,1.0,2.0
2018-01-05,1.0,1.0
2018-01-07,0.0,3.0
2018-01-08,2.0,1.0
2018-01-09,0.0,1.0
2018-01-10,0.0,3.0
2018-01-12,0.0,1.0
2018-01-13,1.0,0.0


## Save down sentiment time series

In [14]:
start_date = '2018-01-01'
end_date = '2019-08-10'

#### Run this only for new tickers

In [16]:
# ticker_list = get_all_stocktwits_tickers()
ticker_list = [
    'UBER',  # Uber
    'LYFT',  # Lyft
    'WORK',  # Slack
    'ZM',  # Zoom
    'PINS',  # Pinterest
    'CHWY',  # Chewy
    'CRWD',  # Crowdstrike
]

In [17]:
root_dir = stock_twits_reader.get_root_dir()
save_dir = os.path.join(root_dir, 'processed', 'sentiment_summary_prelim')
save_dir

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim'

In [18]:
save_file_format = '{ticker}_sentiment_summary.pkl'
save_path_format = os.path.join(save_dir, save_file_format)
save_path_format

'/Users/seung-jae_bang/Personal/Research/Stock_Sentiment/data/Stocktwits/processed/sentiment_summary_prelim/{ticker}_sentiment_summary.pkl'

In [19]:
for t in ticker_list:
    tic = time.time()
    
    raw_df = stock_twits_reader.read_twit_file_in_range(t, start_date, end_date, cols='default')
    daily_sentiment_df = get_daily_sentiment_count_df(raw_df)
    daily_sentiment_df.to_pickle(save_path_format.format(ticker=t))
    
    toc = time.time()
    print('Run for {} took {} mins'.format(t, (toc - tic) / 60))

Run for UBER took 0.18884118795394897 mins
Run for LYFT took 0.224221932888031 mins
Run for WORK took 0.10459887981414795 mins
Run for ZM took 0.10971521536509196 mins
Run for PINS took 0.1195593516031901 mins
Run for CHWY took 0.0748002012570699 mins
Run for CRWD took 0.06573506593704223 mins
